# BONDAD DE AJUSTE

## Test de Bondad de Ajuste con el test $\chi^2$ de Pearson

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats

In [3]:
population = pd.DataFrame(['white']*1000000+['hispano']*600000
                          +['black']*500000+['asian']*150000
                         + ['other']*350000)

In [4]:
valencia = pd.DataFrame(['white']*600+['hispano']*300+
                       ['black']*250 + ['asian'] * 75 +
                        ['other']*150)

In [5]:
population_table = pd.crosstab(index = population[0], columns = "count")
valencia_table = pd.crosstab(index = valencia[0], columns = "count")

In [6]:
print(population_table)

col_0      count
0               
asian     150000
black     500000
hispano   600000
other     350000
white    1000000


In [7]:
print(valencia_table)

col_0    count
0             
asian       75
black      250
hispano    300
other      150
white      600


$$\chi_0 = \sum_{i-1}^n \frac{(o_i - e_i)^2}{e_i} \sim\chi^2_{n-1-k}$$

Donde $n$ es el número de clases o categorías, $o_i$ son las frecuencias obsrvadas y $e_i$ son las frecuencias teóricas y por último $k$ es el número de parámetros estimados para la distribución teórica

In [8]:
o_i = valencia_table
o_i

col_0,count
0,
asian,75
black,250
hispano,300
other,150
white,600


In [9]:
n = len(valencia)
n

1375

In [10]:
p_i = population_table/len(population)
print(p_i)
print('=' * 100)
print(sum(p_i['count']))

col_0       count
0                
asian    0.057692
black    0.192308
hispano  0.230769
other    0.134615
white    0.384615
1.0


In [11]:
e_i = n * p_i
e_i

col_0,count
0,
asian,79.326923
black,264.423077
hispano,317.307692
other,185.096154
white,528.846154


In [12]:
chi_0 = (((o_i - e_i)**2)/e_i).sum()
chi_0

col_0
count    18.194805
dtype: float64

### Comprobación con valor cŕitico de la distribución

In [13]:
alpha = 0.05
crit = stats.chi2.ppf(q = 1-alpha, df = len(o_i)-1)
crit

9.487729036781154

In [14]:
if (chi_0 > crit).bool:
    print("Rechazamos H0")
else:
    print("NO hay razones para rechazar chi_0")

Rechazamos H0


$$p = P(\chi^2_{k-1} > \chi_0) = 1 - F_{\chi^2_{k-1}}{\chi_0}$$

In [16]:
p_val = 1 - stats.chi2.cdf(x = chi_0, df = len(o_i)-1)
p_val

array([0.00113047])

In [17]:
if p_val < alpha:
    print('Rechazamos HO')
else:
    print('No hay evidencias para rechazar HO')

Rechazamos HO


## Automatizamos el código con el test de $\chi^2$ de ```scipy.stats```

In [20]:
stats.chisquare(f_obs = o_i, f_exp = e_i)

Power_divergenceResult(statistic=array([18.19480519]), pvalue=array([0.00113047]))

**Otra forma de hacer el test de $\chi^2$**

In [26]:
table = [[10, 20, 30, 40], [6, 9, 15, 22]]
esta, pv, df, expected = stats.chi2_contingency(table)

In [28]:
print("Estadisctico = {}, pvalue = {}, df = {}".format(esta, pv, df))

Estadisctico = 0.2669461795157015, pvalue = 0.9661202118012747, df = 3


In [29]:
print(expected)

[[10.52631579 19.07894737 29.60526316 40.78947368]
 [ 5.47368421  9.92105263 15.39473684 21.21052632]]


In [46]:
tabla = [[o_i.values], [e_i.values]]
esta, pv, df, expected = stats.chi2_contingency(tabla)

In [47]:
print("Estadisctico = {}, pvalue = {}, df = {}".format(esta, pv, df))

Estadisctico = 9.171739568069004, pvalue = 0.056947264254736917, df = 4


In [48]:
tabla

[[array([[ 75],
         [250],
         [300],
         [150],
         [600]])],
 [array([[ 79.32692308],
         [264.42307692],
         [317.30769231],
         [185.09615385],
         [528.84615385]])]]

# TEST DE NORMALIDAD

In [49]:
data = [0.873, 0.121, 2.817, -0.945, -0.55, -1.436, 0.36, -1.478, -1.869, -1.637]



$$
\left\{
\begin{array}{ll}
H_0: & \mathrm{la\ muestra\ proviene\ de\ una\ normal}\\\
H_1: & \mathrm{la\ muestra\ no\ proviene\ de\ una\ normal}
\end{array}
\right.
$$



### Test de Shapiro-Wilk

* Cada Observación de la meustra debe ser independiente identicamente distribuidas (idd)

In [52]:
esta, pv = stats.shapiro(data)
print("Estadisctico = {}, pvalue = {}".format(esta, pv))
if pv > 0.05:
    print("Es probablemente una muestra procedente de una Normal")
else:
    print("No parece que proceda de una Normal")

Estadisctico = 0.8887019157409668, pvalue = 0.16395507752895355
Es probablemente una muestra procedente de una Normal


### Test d'Agostino $K^2$
* Cada Observación de la meustra debe ser independiente identicamente distribuidas (idd)
* $$n\geq20$$

In [54]:
stat, pv = stats.normaltest(data)
print("Estadisctico = {}, pvalue = {}".format(esta, pv))
if pv > 0.05:
    print("Es probablemente una muestra procedente de una Normal")
else:
    print("No parece que proceda de una Normal")

Estadisctico = 0.8887019157409668, pvalue = 0.11628038249613654
Es probablemente una muestra procedente de una Normal


/home/guillem/anaconda3/envs/masterds/lib/python3.7/site-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  "anyway, n=%i" % int(n))


### Test de Anderson-Darling

* Cada Observación de la meustra debe ser independiente identicamente distribuidas (idd)

In [58]:
result = stats.anderson(data)
result

AndersonResult(statistic=0.43790621884128633, critical_values=array([0.501, 0.57 , 0.684, 0.798, 0.95 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]))

In [62]:
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < cv:
        print("Probablemente Gaussiana a un nivel de significación del {}%". format(sl))
    else:
        print("Probablemente no Gaussiana a un nivel de significación del {}%".format(sl))

Probablemente Gaussiana a un nivel de significación del 15.0%
Probablemente Gaussiana a un nivel de significación del 10.0%
Probablemente Gaussiana a un nivel de significación del 5.0%
Probablemente Gaussiana a un nivel de significación del 2.5%
Probablemente Gaussiana a un nivel de significación del 1.0%
